In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import json
import os
import sys
import pandas as pd
import getpass
import pymysql

### Defining functions

In [30]:
def getUsersInfo(cur,user):
    cur.execute("SELECT * FROM " + user)
    feats = []
    for feat in cur.fetchall():
        feats.append(feat)

    featnames = []
    cur.execute("DESCRIBE " + user)
    for col in cur.fetchall():
        if col[0] == 'id':
            continue
        featnames.append(col[0])

    return feats, featnames
    
def createNodes(feats,featnames,g,color):
  
    for node in feats:
        label = node[0]
        g.add_node(label)
        size = len(node)
        for i in range(1,size):        
            g.nodes[label]['color'] = color
        k = 1
        for featname in featnames:
            g.nodes[label][featname] = node[k]
            k += 1

def createEdges(cur,g):
    
    cur.execute("SELECT x,y FROM edge")
    edges = []

    for e in cur.fetchall():
        edges.append(e)
    
    for e in edges:
        g.add_edge(e[0],e[1])



### Creating the DB

In [23]:
# Establishing connection with the server
p = getpass.getpass()
connection = pymysql.connect(host='localhost', port=3306,user='root',passwd = p)
cur = connection.cursor()

cur.execute("CREATE DATABASE facebook;")

In [24]:
# Filled database using MySQL Workbench.

cur.execute("USE facebook;")
cur.execute("SHOW TABLES;")
for t in cur.fetchall():
    print(t)

('edge',)
('user0',)
('user1',)
('user2',)
('user3',)


### Formating feats and featnames

In [25]:
feats = [[]] * 4
featnames = [[]] * 4

for i in range(4):
    feats[i], featnames[i] = getUsersInfo(cur,"user" + str(i))

### Closing connection

In [22]:
cur.close()
connection.close()

### Building the graph

In [31]:
# One color for each dataset:
colors = ['red','blue','chocolate','forestgreen']

# G will contain info from all datasets.
G = nx.Graph()

for i in range(4):
    createNodes(feats[i],featnames[i],G,colors[i])
    createEdges(cur,G)

### Plotting G (sketch)

In [34]:
### PLEASE SEE THE IMAGE IN THE MAIN FOLDER ###

#posG = nx.random_layout(G)  # positions for all nodes
#
#labels = {}
#for n in G.nodes:
#    labels[n] = n
#
# 
#nx.draw_networkx(G,
#                 pos=posG,
#                 label="Social Circles",
#                 labels=labels,
#                 with_labels=True,
#                 font_size=0.5,
#                 width=0.01,
#                 node_color= [nx.get_node_attributes(G,'color')[g] for g in G.nodes],
#                 node_size=3
#                 )
#
#plt.savefig("socialCirclesSketch.jpg",format="jpg",dpi=700)
#plt.show()

### Converting the graph to Json

In [7]:
with open('./dataset/graphs.json', 'w') as f:
    f.write(json.dumps(nx.cytoscape_data(G)))

### Deploying on the web

In [9]:
import urllib.request
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_cytoscape as cyto
from dash.dependencies import Input, Output

app = dash.Dash(__name__)
app.title = "Social Circles"
server = app.server

app.scripts.config.serve_locally = True
app.css.config.serve_locally = True


with open('./dataset/graphs.json', 'r') as f:
    data = json.loads(f.read())
    
with open('./styles/cy-style.json') as f:
    stylesheet = json.loads(f.read())

graph = data['elements']

styles = {
    'container': {
        'position': 'fixed',
        'display': 'flex',
        'flex-direction': 'row',
        'height': '100%',
        'width': '100%',
        'font-family': 'arial, sans-serif'
    },
    'cy-container': {
        'flex': '1',
        'position': 'relative'
    },
    'cytoscape': {
        'position': 'absolute',
        'width': '100%',
        'height': '100%',
        'z-index': 999
    },
    'responsive': {
        'margin-right': '30px',
        'height': '10%'
    },
    'list': {
        'list-style': 'none',
        'text-align': 'left'
    }
}
    
# App
app.layout = html.Div(style=styles['container'], children=[
    html.Div([
        html.Div("Interpretation", style={
            'text-align': 'center',
            'font-size': '20', 
            'font-weight': 'bolder',
            'margin-bottom': '20px',
            'margin-top': '30vh'
            }),
        html.Div("• The higher the opacity/size, the more friends", style={
            'font-size': '20', 
            'margin-bottom': '10px'
            }),
        html.Div("• Genders :", style={
            'font-size': '20', 
            }),
        html.Ul(children=[
            html.Li("🔺:  Undefined"),
            html.Li("🟡:    Female"),
            html.Li("🟦:    Male"),
        ], style=styles['list']),
         html.Div("• Isolated Graph = social bubble", style={
            'font-size': '20',
            }),
    ], style={'border-right': '1px solid black', 'padding': '5px'}),
    html.Div(className='cy-container', style=styles['cy-container'], children=[
        cyto.Cytoscape(
            id='cytoscape',
            elements=graph,
            stylesheet=stylesheet,
            style=styles['cytoscape'],
            layout={
                'name': 'cose',
                'idealEdgeLength': 100,
                'nodeOverlap': 100,
                'refresh': 20,
                'fit': True,
                'padding': 30,
                'randomize': False,
                'componentSpacing': 100,
                'nodeRepulsion': 40000000,
                'edgeElasticity': 100,
                'nestingFactor': 5,
                'gravity': 80,
                'numIter': 1000,
                'initialTemp': 800,
                'coolingFactor': 0.95,
                'minTemp': 1.0
            },
            responsive=True
        )
    ])
])

@app.callback(Output('cytoscape', 'responsive'), [Input('toggle-button', 'n_clicks')])
def toggle_responsive(n_clicks):
    n_clicks = 2 if n_clicks is None else n_clicks
    toggle_on = n_clicks % 2 == 0
    return toggle_on

@app.callback(Output('toggle-text', 'children'), [Input('cytoscape', 'responsive')])
def update_toggle_text(responsive):
    return '\t' + 'Responsive ' + ('On' if responsive else 'Off')

app.css.config.serve_locally = True
app.scripts.config.serve_locally = True

if __name__ == '__main__':
    app.run_server()